In [1]:
import torchvision.models as models


vgg16 = models.vgg16(pretrained=True)

C:\Users\sibghi\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\sibghi\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms

In [2]:
data_dir = 'C:/Users/sibghi/Downloads/Post-hurricane'

train = datasets.ImageFolder(data_dir + "/train_another",transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

#Validation
validation = datasets.ImageFolder(data_dir + "/validation_another",transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

In [3]:
#Chargement en DataLoader avec batch 128 pour éviter les crashs en entrainement et shuffle
#pour mélanger les batchs à chaque étape de l'entrainement
train_dl = torch.utils.data.DataLoader(train, 20, shuffle = True, num_workers = 4, pin_memory = True)

val_dl = torch.utils.data.DataLoader(validation, 20, shuffle = True,num_workers = 4, pin_memory = True)

In [4]:
original_dataset_dir = 'C:/Users/sibghi/Downloads/Post-hurricane'


In [5]:


train_damage_dir = original_dataset_dir + '/train_another/damage'
validation_damage_dir = original_dataset_dir + '/validation_another/damage'
test_damage_dir = original_dataset_dir + '/test_another/damage'

train_nodamage_dir = original_dataset_dir + '/train_another/no_damage'
validation_nodamage_dir = original_dataset_dir + '/validation_another/no_damage'
test_nodamage_dir = original_dataset_dir + '/test_another/no_damage'



In [6]:
print('total training damage images: ',len(os.listdir(train_damage_dir)))
print('total validation damage images: ',len(os.listdir(validation_damage_dir)))
print('total test damage images: ',len(os.listdir(test_damage_dir)))

print('total training no damage images: ',len(os.listdir(train_nodamage_dir)))
print('total validation no damage images: ',len(os.listdir(validation_nodamage_dir)))
print('total test no damage images: ',len(os.listdir(test_nodamage_dir)))

total training damage images:  5000
total validation damage images:  1000
total test damage images:  8000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


In [7]:

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))


In [8]:

class ConvNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(32, 64, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(64, 128, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 128, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(128*7*7, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.Sigmoid())

    def forward(self, xb):
        return self.network(xb)

model = ConvNet()

In [9]:
model

ConvNet(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=6272, out_features=512, bias=True)
    (14): ReLU()
    (15): Linear(in_features=512, out_features=2, bias=True)
    (16): Sigmoid()
  )
)

In [9]:

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    step = 0
    outputs = []
    for batch in val_loader :
        step += 1
        outputs.append(model.validation_step(batch))
        if step == 50:
                break 
                
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        step = 0
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            step += 1
            if step == 100:
                break 
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history, model

In [ ]:
#num_epochs = 50
#opt_func = torch.optim.RMSprop
#lr = 1e-4
#fitting the model on training data and record the result after each epoch
#history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [ ]:
#reproduire le 1er modele 
#data augmentation + RMSprop optimizer 
num_epochs = 50
opt_func = torch.optim.RMSprop
lr = 1e-4
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [14]:
class ConvNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(32, 64, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(64, 128, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 128, kernel_size =(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.Linear(128*7*7, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.Sigmoid())

    def forward(self, xb):
        return self.network(xb)

model2 = ConvNet()

In [11]:
#reproduire le meilleur modele
#data augmentation et dropout + Adam optimizer 
num_epochs = 50
opt_func = torch.optim.Adam
lr = 1e-4
history2, model2 = fit(num_epochs, lr, model2, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 0.6795, val_loss: 0.6194, val_acc: 0.7420
Epoch [1], train_loss: 0.5881, val_loss: 0.5261, val_acc: 0.7940
Epoch [2], train_loss: 0.5428, val_loss: 0.5441, val_acc: 0.7520
Epoch [3], train_loss: 0.5307, val_loss: 0.5135, val_acc: 0.8010
Epoch [4], train_loss: 0.5078, val_loss: 0.4916, val_acc: 0.8120
Epoch [5], train_loss: 0.4985, val_loss: 0.4643, val_acc: 0.8470
Epoch [6], train_loss: 0.4549, val_loss: 0.4546, val_acc: 0.8570
Epoch [7], train_loss: 0.4654, val_loss: 0.4065, val_acc: 0.9070
Epoch [8], train_loss: 0.4244, val_loss: 0.4425, val_acc: 0.8690
Epoch [9], train_loss: 0.4300, val_loss: 0.4072, val_acc: 0.9100
Epoch [10], train_loss: 0.4220, val_loss: 0.4077, val_acc: 0.9120
Epoch [11], train_loss: 0.4097, val_loss: 0.4220, val_acc: 0.8870
Epoch [12], train_loss: 0.4142, val_loss: 0.4049, val_acc: 0.9060
Epoch [13], train_loss: 0.4108, val_loss: 0.4261, val_acc: 0.8840
Epoch [14], train_loss: 0.4106, val_loss: 0.3969, val_acc: 0.9220
Epoch [15], train_lo

In [12]:
#!pip install torchinfo
from torchinfo import summary

In [17]:
summary(model2)


Layer (type:depth-idx)                   Param #
ConvNet                                  --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       896
│    └─ReLU: 2-2                         --
│    └─MaxPool2d: 2-3                    --
│    └─Conv2d: 2-4                       18,496
│    └─ReLU: 2-5                         --
│    └─MaxPool2d: 2-6                    --
│    └─Conv2d: 2-7                       73,856
│    └─ReLU: 2-8                         --
│    └─MaxPool2d: 2-9                    --
│    └─Conv2d: 2-10                      147,584
│    └─ReLU: 2-11                        --
│    └─MaxPool2d: 2-12                   --
│    └─Flatten: 2-13                     --
│    └─Dropout: 2-14                     --
│    └─Linear: 2-15                      3,211,776
│    └─ReLU: 2-16                        --
│    └─Linear: 2-17                      1,026
│    └─Sigmoid: 2-18                     --
Total params: 3,453,634
Trainable params: 3,453

In [21]:
torch.save(model2.state_dict(), "C:/Users/sibghi/Téléchargements")

In [49]:
import torchvision.models as models

vgg19 = models.vgg19(pretrained=True)

In [47]:
vgg19.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [50]:
# Freeze model weights
for param in vgg19.parameters():
    param.requires_grad = False
    
# Add on classifier in the last layer 
vgg19.classifier[6] = nn.Sequential(
                      nn.Linear(512, 2),
                      nn.Sigmoid())


In [51]:
vgg19.classifier


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=512, out_features=2, bias=True)
    (1): Sigmoid()
  )
)

In [52]:
#reproduire le meilleur modele
#pretrainmodel with sequential in last layer + Adam optimizer 
num_epochs = 30
opt_func = torch.optim.Adam
lr = 1e-4
history3, model3 = fit(num_epochs, lr, vgg19, train_dl, val_dl, opt_func)

AttributeError: 'VGG' object has no attribute 'training_step'

In [26]:
resnet50 = models.resnet50(pretrained=True)

C:\Users\sibghi\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\sibghi/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [38]:
summary(resnet50)

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            (9,408)
├─BatchNorm2d: 1-2                       (128)
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  (4,096)
│    │    └─BatchNorm2d: 3-2             (128)
│    │    └─Conv2d: 3-3                  (36,864)
│    │    └─BatchNorm2d: 3-4             (128)
│    │    └─Conv2d: 3-5                  (16,384)
│    │    └─BatchNorm2d: 3-6             (512)
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              (16,896)
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  (16,384)
│    │    └─BatchNorm2d: 3-10            (128)
│    │    └─Conv2d: 3-11                 (36,864)
│    │    └─BatchNorm2d: 3-12            (128)
│    │    └─C

In [37]:
for param in vgg19.parameters():
    param.requires_grad = False

vgg19.classifier = nn.Sequential(nn.Linear(512, 2),
            nn.Sigmoid())  

TypeError: __init__() got an unexpected keyword argument 'inplace'

In [29]:
conv_base = resnet50.conv1 # remove the last maxpooling layer
conv_base

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [22]:
#!pip install torchinfo

Defaulting to user installation because normal site-packages is not writeable


In [19]:

from torchvision.models import resnet50, ResNet50_Weights

# Initialize model
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\sibghi/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [24]:
def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count,4,4,512))
    labels = np.zeros(shape = (sample_count))
    generator = datagen.flow_from_directory(
                    directory, 
                    target_size = (150,150),
                    batch_size = batch_size,
                    class_mode = 'binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count: #generators yield data indefinitely
            break                          #have to break after we have seen every image once
    return features, labels



In [ ]:


train_features, train_labels = extract_features(train_dir, 10000)
validation_features, validation_labels = extract_features(validation_dir, 2000)
test_features, test_labels = extract_features(test_dir, 2000)
#the extracted features are of shape (sample_count, 4, 4, 512), we must flatten them to (sample_count, 8192)
train_features = np.reshape(train_features, (10000, 4*4*512))
validation_features = np.reshape(validation_features, (2000, 4*4*512))
test_features = np.reshape(test_features, (2000, 4*4*512))

